<a href="https://colab.research.google.com/github/paryoja/deeplearningstudy/blob/master/Regression_(2)_2_dimensional_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습

2차원 데이터의 linear regression을 해보자

입력은 dx1, dx2

맞춰야 하는 값은 dy 이다

In [0]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

_x1 = np.arange(20, dtype=np.float32)
_x2 = np.arange(20, dtype=np.float32)

dx1 = []
dx2 = []
dy = []
for x1 in _x1:
    for x2 in _x2:
        dx1.append(x1)
        dx2.append(x2)
        dy.append(x1 * 3 + x2 * 5 - 1 + np.random.normal(0, 5))

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(dx1, dx2, dy, c='r')
plt.show()


# 답안

먼저 어느 함수를 통해서 regression을 할 것인지를 정해야한다.

w1, w2, b 의 변수를 이용해서

w1 * x1 + w2 * x2 + b 라는 함수로 예측해보자

In [0]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([1]), name='w1')
w2 = tf.Variable(tf.random_normal([1]), name='w2')
b = tf.Variable(tf.zeros([1]), name='b')

x1 = tf.constant(dx1)
x2 = tf.constant(dx2)
y = tf.constant(dy)

# 모델을 만드는 핵심 부분
hypothesis = w1 * x1 + w2 * x2 + b

# loss는 squared error sum을 사용했다
loss = tf.reduce_sum(tf.square(hypothesis - y))

learning_rate = 0.00001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(100):
  # _ 도 합법적인 변수의 이름이다.
  # 파이썬에서는 무시할 변수가 있다면 그 변수의 이름 _로 한다
  # train_op 의 sess.run 결과는 None이기 때문에 굳이 필요 없다
  l, _ = sess.run([loss, train_op])
  
  print('epoch',i, 'loss', l)

w1_value, w2_value, b_value = sess.run([w1, w2, b])

prediction = w1_value * dx1 + w2_value * dx2 + b_value


if np.isnan(prediction[0]):
  print("Prediction is Nan!!")
else:  
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  ax.scatter(dx1, dx2, dy, c='r')
  ax.scatter(dx1, dx2, prediction)
  plt.show()


Learning rate 에 따른 수렴속도 차이를 그려보자



In [0]:
import tensorflow as tf

w1 = tf.Variable(tf.random_normal([1]), name='w1')
w2 = tf.Variable(tf.random_normal([1]), name='w2')
b = tf.Variable(tf.zeros([1]), name='b')

x1 = tf.constant(dx1)
x2 = tf.constant(dx2)
y = tf.constant(dy)

hypothesis = w1 * x1 + w2 * x2 + b

loss = tf.reduce_sum(tf.square(hypothesis - y))


# 비교할 learning rate의 리스트
learning_rate_list = [0.00001, 0.000001, 0.0000001]

# 각 learning_rate 마다 loss의 변화를 기록할 리스트
loss_per_rate = []

for learning_rate in learning_rate_list:
  #각 learning_rate 마다 optimizer를 새로 만들어줘야함
  
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss)

  sess = tf.Session()
  
  # random 함수를 이용하므로 돌릴때마다 초기 값이 달라지는 것을 막기 위해 
  # 같은 seed 값으로 설정
  tf.set_random_seed(1000)

  # 실제 초기화가 되는 부분
  sess.run(tf.global_variables_initializer())
  
  # 같은 값으로 됬는지 확인 하기 위해 찍어봄
  print(sess.run([w1, w2, b]))
  
  # 이 learning rate에서의 loss 의 변화를 기록할 리스트
  loss_list = []
  
  for i in range(100):
    # _ 도 합법적인 변수의 이름이다.
    # 파이썬에서는 무시할 변수가 있다면 그 변수의 이름 _로 한다
    # train_op 의 sess.run 결과는 None이기 때문에 굳이 필요 없다
    
    l, _ = sess.run([loss, train_op])
    
    # 계산된 loss값을 저장한다.
    loss_list.append(l)

  w1_value, w2_value, b_value = sess.run([w1, w2, b])

  loss_per_rate.append(loss_list)

  if np.isnan(prediction[0]):
    print("Prediction is Nan!!")
    break

# 사용할 color의 리스트
color_list = ['r', 'g', 'b']

# 파이썬 내장함수인 zip과 enumerate를 사용했는데, 
# 익숙하지 않다면 인터넷 검색을 해서 사용법을 익히도록 하자
# 매우 빈번히 쓰이는 함수이다

for i, (lr, color) in enumerate(zip(learning_rate_list, color_list)):
  print(lr, color)
  plt.plot(range(100), loss_per_rate[i], color, label=lr)

plt.legend(loc='upper right')
  

# Matrix 사용하기


hypothesis = w1 * x1 + w2 * x2 + b 를 벡터 표현으로 써보자

W를 (1, 2) 행렬 

```
[[w1, w2]]
```

x를 열벡터 
```
[x1,
 x2]
```

라고 하면
W 와 x의 행렬곱은

 w1 * x1 + w2 * x2 가된다
 
 즉,
 
 hypothesis는 W x + b 
 
 로 표현 가능하며,
 
 이렇게 행렬과 벡터의 곱으로 구성할때, 아무런 연산자 표시가 없으면 행렬곱을 의미한다.
 
 


W를 


```
[[w1, w2, b]]
```

x를 열벡터 
```
[x1,
 x2,
 1]
```
로 표현한다면,


hypothesis 를 W x 로 표시하는 것도 가능하다.


In [0]:
import tensorflow as tf


x = tf.constant([dx1, dx2]) 

# dx1은 (400, ), dx2는 (400, ) 이므로, (400, ) 가 두개가 들어 있는 matrix 
# 즉, (2, 400) 이다

W = tf.Variable(tf.random_normal([1,2]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')


y = tf.constant(dy)

# 다음 연산 결과는 1, 400 이다
hypothesis = tf.matmul(W, x) + b

loss = tf.reduce_sum(tf.square(hypothesis - y))

learning_rate = 0.00001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)


sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 실행하기 전에 위에서 설명한대로 x의 shape을 살펴보자
# 텐서의 shape을 보기위해서는 tf.shape 함수를 사용한다

print('shape of x', sess.run(tf.shape(x)))


for i in range(100):
  l, _ = sess.run([loss, train_op])
  
  # print('epoch',i, 'loss', l)

W_value, b_value = sess.run([W, b])

# numpy에서도 matrix multiplication은 matmul이다
prediction = np.matmul(W_value,  [dx1, dx2]) + b_value


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(dx1, dx2, dy, c='r')
ax.scatter(dx1, dx2, prediction)
plt.show()


다수의 강의에서는 벡터라 함은,

데이터를 세로로 나열한 column vector를 기준으로 하고 있다.

위에서 사용한 

```
[dx1, dx2]
```
를 찍어보면

In [0]:
print(np.array([dx1, dx2]))

잠깐 우리의 데이터를 생각해보자.

데이터 생성은

```
x1 * 3 + x2 * 5 - 1 + np.random.normal(0, 5)
```

의 코드를 통해서 했다.



우리는  (0, 0) 일 때, -1 + noise

(0, 1) 일 때는 4 + noise

(0, 2) 일 때는 9 + noise


값이 나오도록 데이터를 만들었는데,




위 데이터를 보면,  x 값으로 몇을 썼는지 보기가 불편하다.

사람이 코딩 하기에는 





In [0]:
print(np.transpose([dx1, dx2])[:10])

와 같이 row 벡터를 쓰는 것이 더욱 직관적이다.

따라서, 이후 실습에서는 row 벡터를 기준으로 데이터를 표현할 것이다.


달라지는 점은

hypothesis 가 W x 로 계산되는것 이 아니라

x W 로 계산된다.

또한 여기서 W는
```
[[w1]
 [w2]]
```
의 형태로 이루어진다.

row 벡터를 기준으로 구현하면 다음과 같다

In [0]:
import tensorflow as tf
import numpy as np

x = tf.constant(np.transpose([dx1, dx2]))

# dx1은 (400, ), dx2는 (400, ) 이므로, (400, ) 가 두개가 들어 있는 matrix 
# 즉, (2, 400) 이다

W = tf.Variable(tf.random_normal([2, 1]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')


y = tf.constant(dy)

# 다음 연산 결과는 400, 1 이다
hypothesis = tf.matmul(x, W) + b

loss = tf.reduce_sum(tf.square(hypothesis - y))

learning_rate = 0.00000001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)


sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 실행하기 전에 위에서 설명한대로 x의 shape을 살펴보자
# 텐서의 shape을 보기위해서는 tf.shape 함수를 사용한다

print('shape of x', sess.run(tf.shape(x)))


for i in range(10000):
  l, _ = sess.run([loss, train_op])
  if i % 1000 == 0:
    print('epoch',i, 'loss', l)

W_value, b_value = sess.run([W, b])

# numpy에서도 matrix multiplication은 matmul이다
prediction = np.matmul(np.transpose([dx1, dx2]), W_value) + b_value


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(dx1, dx2, dy, c='r')
ax.scatter(dx1, dx2, prediction)
plt.show()


위의 수행 결과를 보면,

loss는 수렴하여 값이 고정 되었으나,

실제 그려보니 전혀 다른 형태임을 보이고 있다.

위에서 문제가 된 버그를  찾아서 고쳐보자

힌트 (뺄셈에서의 broadcasting)

Tensorflow 내에서 transpose를 하기 위해서는

tf.transpose 함수를 이용 할 수 있다.



행 벡터나 열 백터를 transpose하는 것은 reshape 하는 것과 같다

In [0]:
import tensorflow as tf
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline 

_x1 = np.arange(20, dtype=np.float32)
_x2 = np.arange(20, dtype=np.float32)

dx1 = []
dx2 = []
dy = []
for x1 in _x1:
    for x2 in _x2:
        dx1.append(x1)
        dx2.append(x2)
        dy.append(x1 * 3 + x2 * 5 - 1 + np.random.normal(0, 5))

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(dx1, dx2, dy, c='r')
plt.show()


x = tf.constant(np.transpose([dx1, dx2]))

# dx1은 (400, ), dx2는 (400, ) 이므로, (400, ) 가 두개가 들어 있는 matrix 
# 즉, (2, 400) 이다

W = tf.Variable(tf.random_normal([2, 1]), name='W')
b = tf.Variable(tf.zeros([1]), name='b')


y = tf.constant(dy)

# 다음 연산 결과는 400, 1 이다
hypothesis = tf.matmul(x, W) + b

loss = tf.reduce_sum(tf.square(hypothesis - tf.reshape(y, [400,1])))

learning_rate = 0.000001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)


sess = tf.Session()
sess.run(tf.global_variables_initializer())


# 실행하기 전에 위에서 설명한대로 x의 shape을 살펴보자
# 텐서의 shape을 보기위해서는 tf.shape 함수를 사용한다

print('shape of x', sess.run(tf.shape(x)))


for i in range(10000):
  l, _ = sess.run([loss, train_op])
  if i % 1000 == 0:
    print('epoch',i, 'loss', l)

W_value, b_value = sess.run([W, b])

# numpy에서도 matrix multiplication은 matmul이다
prediction = np.matmul(np.transpose([dx1, dx2]), W_value) + b_value


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(dx1, dx2, dy, c='r')
ax.scatter(dx1, dx2, prediction)
plt.show()
